In [1]:
import pandas as pd
import numpy as np
from collections import Counter
%matplotlib inline

In [2]:
feature_table = pd.read_csv("170728_2015-01-01_alphasort.csv")

In [3]:
tmp = feature_table[feature_table.feature_importance>0.00125]
tmp.sort_values(by='feature_importance',ascending=False,inplace=True)
tmp.reset_index(drop=True,inplace=True)
top_feats = list(tmp.feature)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
top_feats

['dispatch_id_p3m_dispatchinitiatiationtype_ci_sum',
 'dispatch_id_p6m_dispatchinitiatiationtype_ci_sum',
 'ocag_id_all_officerage_max',
 'dispatch_id_p3m_dispatchtype_domestic_disturb_sum',
 'dispatch_id_p1m_dispatchinitiatiationtype_ci_sum',
 'ts_id_p3m_trafficstopsbyrace_black_sum',
 'dispatch_id_p3m_dispatchtype_traffic_sum',
 'dispatch_id_p3m_dispatchtype_disorder_sum',
 'dispatch_id_p3m_dispatchtype_theft_sum',
 'dispatch_id_p6m_dispatchtype_assist_sum',
 'dispatch_id_p1y_dispatchinitiatiationtype_ci_sum',
 'ts_id_p3m_trafficstops_count',
 'dispatch_id_p3m_dispatchtype_assist_sum',
 'dispatch_id_p6m_dispatchtype_disorder_sum',
 'ts_id_p1m_trafficstops_count',
 'dispatch_id_p1m_dispatchtype_traffic_sum',
 'demarrests_id_p5y_populationdensity_avg',
 'ts_id_p3m_trafficstopsbystopresult_verbal_warning_sum',
 'dispatch_id_p6m_dispatchtype_domestic_disturb_sum',
 'dispatch_id_p1m_dispatchtype_disorder_sum',
 'dispatch_id_p6m_dispatchtype_assault_sum',
 'dispatch_id_p6m_dispatchtype_sus

In [5]:
train_matrix = pd.read_csv("170728_2015-01-01_train-matrix-and-labels.csv")
labels = train_matrix.outcome
del train_matrix['outcome']
#train_matrix = train_matrix.loc[:,top_feats]

In [6]:
directionality = []
for feature in feature_table.feature:
    tmp = train_matrix[feature]
    pos = tmp[labels==1]
    neg = tmp[labels==0]
    if np.mean(pos) > np.mean(neg):
        directionality.append('pos')
    elif np.mean(pos) < np.mean(neg):
        directionality.append('neg')
    elif np.mean(pos) == np.mean(neg):
        directionality.append('equal')
    else:
        directionality.append('wtf')

In [7]:
feature_table['direction'] = directionality

In [8]:
numvals = []
for feature in feature_table.feature:
    n = len(train_matrix[feature].value_counts())
    numvals.append(n)

In [9]:
feature_table['numvals'] = numvals

### grabbin feats

In [10]:
qstr = "record_type=='ocnd' & numvals > 1"
ocnds = list(feature_table.query(qstr).feature)

In [11]:
qstr = "record_type=='arrests' & direction=='pos' & temp_agg=='p5y' & num_agg=='sum'"
bad_arrests = list(feature_table.query(qstr).feature)

In [12]:
qstr = "record_type=='dispatch' & direction=='pos' & temp_agg=='p5y' & num_agg=='sum'"
bad_dispatches = list(feature_table.query(qstr).feature)

In [13]:
qstr = "record_type=='ts' & direction=='pos' & temp_agg=='p5y' & num_agg=='sum'"
bad_ts = list(feature_table.query(qstr).feature)

In [14]:
qstr = "record_type=='ir' & direction=='pos' & temp_agg=='p5y' & num_agg=='sum'"
bad_ir = list(feature_table.query(qstr).feature)

In [15]:
qstr = "record_type=='ic' & direction=='pos' & temp_agg=='p5y' & num_agg=='sum'"
bad_ic = list(feature_table.query(qstr).feature)

### Let's start measuring!

In [16]:
from scipy.stats import histogram as hist

def minmax(matrix,feat):
    tmp = matrix.loc[:,feat]
    return min(tmp),max(tmp)

def probnorm(array):
    divisor = sum(array)
    return np.asarray([item / float(divisor) for item in array])

#### Officer demographics

In [17]:
def euc_binary(ocnd,feat):
    featlabel0 = probnorm(hist(train_matrix[feat][labels==0],100,minmax(train_matrix,feat))[0])
    featlabel1 = probnorm(hist(train_matrix[feat][labels==1],100,minmax(train_matrix,feat))[0])
    featdist = np.linalg.norm(featlabel0-featlabel1)
    
    ocnd0 = train_matrix[feat][train_matrix[ocnd]==0]
    ocnd1 = train_matrix[feat][train_matrix[ocnd]==1]
    
    ocnd0label0 = probnorm(hist(ocnd0[labels==0],100,minmax(train_matrix,feat))[0])
    ocnd0label1 = probnorm(hist(ocnd0[labels==1],100,minmax(train_matrix,feat))[0])
    ocnd1label0 = probnorm(hist(ocnd1[labels==0],100,minmax(train_matrix,feat))[0])
    ocnd1label1 = probnorm(hist(ocnd1[labels==1],100,minmax(train_matrix,feat))[0])
    
    ocnd0dist = np.linalg.norm(ocnd0label0-ocnd0label1)
    ocnd1dist = np.linalg.norm(ocnd1label0-ocnd1label1)
    
    return featdist,ocnd0dist,ocnd1dist

In [75]:
"""
main_feat = []
featdist = []
demog_feat = []
demog0dist = []
demog1dist = []

counter = 0
for top_feat in feature_table.feature:
    for ocnd in ocnds:
        if top_feat!=ocnd:
            counter+=1
            main_feat.append(top_feat)
            demog_feat.append(ocnd)
            
            dists = euc_binary(ocnd,top_feat)
            featdist.append(dists[0])
            demog0dist.append(dists[1])
            demog1dist.append(dists[2])
            
            if np.random.binomial(1,0.001)==1:
                print counter
                
            
df = pd.DataFrame({
        "main_feat":main_feat,
        "demog_feat":demog_feat,
        "featdist":featdist,
        "demog0dist":demog0dist,
        "demog1dist":demog1dist
    })
    
#df['zerodiff'] = [abs(df.featdist.loc[i]-df.demog0dist.loc[i]) for i in range(len(df))]
#df['onediff'] = [abs(df.featdist.loc[i]-df.demog1dist.loc[i]) for i in range(len(df))]
#df['zerosize'] = [train_matrix[item].value_counts()[0] for item in df.demog_feat]

dfsub = feature_table[['feature','feature_importance']]
dfsub.rename(columns={"feature":"main_feat"},inplace=True)
dfsub.set_index('main_feat',inplace=True)
df = df.join(dfsub,on='main_feat')
df.to_csv("170728_2015-01-01_interactions_topfeat_ocnd_probnorm.csv",index=False)
"""

'\nmain_feat = []\nfeatdist = []\ndemog_feat = []\ndemog0dist = []\ndemog1dist = []\n\ncounter = 0\nfor top_feat in feature_table.feature:\n    for ocnd in ocnds:\n        if top_feat!=ocnd:\n            counter+=1\n            main_feat.append(top_feat)\n            demog_feat.append(ocnd)\n            \n            dists = euc_binary(ocnd,top_feat)\n            featdist.append(dists[0])\n            demog0dist.append(dists[1])\n            demog1dist.append(dists[2])\n            \n            if np.random.binomial(1,0.001)==1:\n                print counter\n                \n            \ndf = pd.DataFrame({\n        "main_feat":main_feat,\n        "demog_feat":demog_feat,\n        "featdist":featdist,\n        "demog0dist":demog0dist,\n        "demog1dist":demog1dist\n    })\n\ndf.to_csv("170728_2015-01-01_interactions_topfeat_ocnd_probnorm.csv",index=False)\n'

In [76]:
#tmp = df.query("demog0dist > 0.2 & featdist < 0.1 & zerosize > 277")
#tmp = tmp[['demog_feat','main_feat','zerosize','feature_importance']]
#tmp.sort_values(by='feature_importance',ascending=False)
#df.query("demog0dist > 0.2 & featdist < 0.1 & zerosize > 277")

#### Continuous Variables: The Bad

In [56]:
def KLD(p,q):
    p = p + 0.000001 # bc KLD undefined if any bin==0
    q = q + 0.000001
    
    quot = p / q
    logquot = np.apply_along_axis(np.log,0,quot)
    plogquot = p * logquot
    
    return sum(plogquot)

In [57]:
cfeat = "dispatch_id_p6m_dispatchtype_drug_sum"
ifeat = "bad_ic"

In [72]:
featlabel0 = probnorm(hist(train_matrix[cfeat][labels==0],100,minmax(train_matrix,cfeat))[0])
featlabel1 = probnorm(hist(train_matrix[cfeat][labels==1],100,minmax(train_matrix,cfeat))[0])

In [73]:
KLD(featlabel1,featlabel0)

0.13686287398908545

In [75]:
np.log(np.array([2,2,2]))

array([ 0.69314718,  0.69314718,  0.69314718])

In [25]:
def KL_cont(ifeat,cfeat,nbins):
    featlabel0 = probnorm(hist(train_matrix[cfeat][labels==0],100,minmax(train_matrix,cfeat))[0])
    featlabel1 = probnorm(hist(train_matrix[cfeat][labels==1],100,minmax(train_matrix,cfeat))[0])
    
    featdist = 
    
    subdists = []
    for binn in range(nbins):
        sub = train_matrix[cfeat][pd.cut(dfbad[ifeat],bins=nbins,labels=False)==binn]
        sub0 = probnorm(hist(sub[labels==0],10,minmax(train_matrix,cfeat))[0])
        sub1 = probnorm(hist(sub[labels==1],10,minmax(train_matrix,cfeat))[0])
        subdists.append( np.linalg.norm(sub0-sub1) )
    
    return featdist,subdists 

,bad_arrests,bad_ic,bad_ir,bad_dispatches,bad_ts
0,685.0,6.0,16.0,0.0,1101.0
1,405.0,1.0,4.0,0.0,192.0
2,399.0,6.0,22.0,0.0,728.0
3,74.0,0.0,0.0,0.0,97.0
4,224.0,3.0,7.0,0.0,228.0
5,5.0,0.0,0.0,0.0,0.0
6,328.0,2.0,6.0,0.0,452.0
7,339.0,6.0,16.0,0.0,458.0
8,214.0,1.0,4.0,0.0,674.0
9,4.0,1.0,3.0,0.0,2.0


In [18]:
def featagg(featlist,matrix):
    return matrix.loc[:,featlist].sum(axis=1)

In [19]:
bads = {"bad_arrests":bad_arrests,
        "bad_dispatches":bad_dispatches,
        "bad_ts":bad_ts,
        "bad_ir":bad_ir,
        "bad_ic":bad_ic}
dfbad = pd.DataFrame()

In [20]:
for bad in bads.keys():
    dfbad[bad] = featagg(bads[bad],train_matrix)

In [106]:
#dfbad.to_csv("170728_2015-01-01_dfbad.csv",index=False)

In [21]:
def euc_cont(ifeat,cfeat,nbins):
    featlabel0 = probnorm(hist(train_matrix[cfeat][labels==0],10,minmax(train_matrix,cfeat))[0])
    featlabel1 = probnorm(hist(train_matrix[cfeat][labels==1],10,minmax(train_matrix,cfeat))[0])
    featdist = np.linalg.norm(featlabel0-featlabel1)
    
    subdists = []
    for binn in range(nbins):
        sub = train_matrix[cfeat][pd.cut(dfbad[ifeat],bins=nbins,labels=False)==binn]
        sub0 = probnorm(hist(sub[labels==0],10,minmax(train_matrix,cfeat))[0])
        sub1 = probnorm(hist(sub[labels==1],10,minmax(train_matrix,cfeat))[0])
        subdists.append( np.linalg.norm(sub0-sub1) )
    
    return featdist,subdists 

In [22]:
nbins = 4

In [143]:
ifeats = []
cfeats = []
featdists = []
subdists = []

for ifeat in dfbad.columns:
    for cfeat in top_feats:
        ifeats.append(ifeat)
        cfeats.append(cfeat)
        outputs = euc_cont(ifeat,cfeat,nbins)
        featdists.append(outputs[0])
        subdists.append(outputs[1])
        
diffs = pd.DataFrame({
        "ifeat":ifeats,
        "cfeat":cfeats,
        "featdist":featdists,
        "subdist":subdists
    })

In [144]:
diffs.to_csv("/Users/damoncrockett/Desktop/tmptable.csv",index=False)

In [158]:
nbinbs = 4
binn = 3
cfeat = "dispatch_id_p6m_dispatchtype_drug_sum"
ifeat = "bad_ic"

In [159]:
sub = train_matrix[cfeat][pd.cut(dfbad[ifeat],bins=nbins,labels=False)==binn]

In [160]:
sub

4076    27.0
5350    37.0
6692    49.0
8105    43.0
Name: dispatch_id_p6m_dispatchtype_drug_sum, dtype: float64

In [161]:
sub0 = probnorm(hist(sub[labels==0],10,minmax(train_matrix,cfeat))[0])
sub1 = probnorm(hist(sub[labels==1],10,minmax(train_matrix,cfeat))[0])

In [162]:
sub0

array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.25,  0.25,  0.25,  0.  ,  0.  ])

In [163]:
sub1

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan])

In [172]:
int(45.4545)

45

In [164]:
for col in dfbad.columns:
    print pd.cut(dfbad[col],nbins,labels=False).value_counts()

0    8503
1     132
3       5
2       4
Name: bad_arrests, dtype: int64
0    8594
1      43
3       4
2       3
Name: bad_ic, dtype: int64
0    8555
1      82
3       5
2       2
Name: bad_ir, dtype: int64
0    7644
1     952
2      42
3       6
Name: bad_dispatches, dtype: int64
0    8362
1     202
2      60
3      20
Name: bad_ts, dtype: int64
